In [2]:
from astropy.io import fits
import glob, os
from subprocess import call
from astropy.io import fits
import time


os.environ['CRDS_PATH'] = '/Users/zen/crds_cache/jwst_ops'
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['CRDS_CONTEXT'] = 'jwst_1015.pmap'

# jwst
import jwst
from jwst.pipeline import Detector1Pipeline, Spec2Pipeline, Spec3Pipeline
from jwst.cube_build import cube_build_step
from jwst import datamodels
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file

import matplotlib.pyplot as plt
import numpy as n

from spectral_cube import SpectralCube
from scipy import optimize


# Print out what pipeline version we're using
print('JWST pipeline version',jwst.__version__)

JWST pipeline version 1.8.4


In [3]:
def gauss2d(x, y, x0, y0, offset, sigma_x, sigma_y, A):

    x_exp = ( ( x - x0 )**2) / (2 * sigma_x ** 2)
    y_exp = ( ( y - y0 )**2) / (2 * sigma_y ** 2)
    f = A * n.exp( - x_exp - y_exp) + offset

    return f

def image_solve(constants, im):

    x0_val, y0_val, offset_val, sigma_x_val, sigma_y_val, A_val = constants

    y_len, x_len = im.shape
    y_arr = n.arange(0, y_len)
    x_arr = n.arange(0, x_len)
    xx, yy = n.meshgrid(x_arr, y_arr)

    fit = gauss2d(xx, yy, x0_val, y0_val, offset_val, sigma_x_val, sigma_y_val, A_val)

    min_diff = n.nansum(abs(im - fit)**2)

    return min_diff

In [4]:
cache_dir = './'

det1_dir = 'stage1/' # Detector1 pipeline outputs will go here
spec2_dir = 'stage2/' # Spec2 pipeline outputs will go here
spec3_dir = 'stage3/' # Spec3 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if not os.path.exists(det1_dir):
    os.makedirs(det1_dir)
if not os.path.exists(spec2_dir):
    os.makedirs(spec2_dir)
if not os.path.exists(spec3_dir):
    os.makedirs(spec3_dir)

In [5]:
uncal_file_dir = '/Users/zen/projects/JWST_Observations_VHS1256b/reduction/nirspec_a_star/data/'
uncal_files = ['jw01128009001_03106_00001_nrs2_uncal.fits',
              'jw01128009001_03106_00002_nrs2_uncal.fits',
              'jw01128009001_03106_00003_nrs2_uncal.fits',
              'jw01128009001_03106_00004_nrs2_uncal.fits']


for i, file in enumerate(uncal_files):
    uncal_files[i] = uncal_file_dir + uncal_files[i]
print(uncal_files)

print('Found ' + str(len(uncal_files)) + ' input files to process')

['/Users/zen/projects/JWST_Observations_VHS1256b/reduction/nirspec_a_star/data/jw01128009001_03106_00001_nrs2_uncal.fits', '/Users/zen/projects/JWST_Observations_VHS1256b/reduction/nirspec_a_star/data/jw01128009001_03106_00002_nrs2_uncal.fits', '/Users/zen/projects/JWST_Observations_VHS1256b/reduction/nirspec_a_star/data/jw01128009001_03106_00003_nrs2_uncal.fits', '/Users/zen/projects/JWST_Observations_VHS1256b/reduction/nirspec_a_star/data/jw01128009001_03106_00004_nrs2_uncal.fits']
Found 4 input files to process


In [6]:
# Start a timer to keep track of runtime
time0 = time.perf_counter()

In [7]:
for file in uncal_files:
    det1 = Detector1Pipeline() # Instantiate the pipeline
    det1.output_dir = det1_dir # Specify where the output should go
    det1.save_results = True # Save the final resulting _rate.fits files

    
    det1(file)

2022-11-19 20:13:18,092 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-11-19 20:13:18,093 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-11-19 20:13:18,097 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-11-19 20:13:18,098 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-11-19 20:13:18,100 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-11-19 20:13:18,102 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-11-19 20:13:18,104 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2022-11-19 20:13:18,106 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2022-11-19 20:13:18,108 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-11-19 20:13:18,109 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-11-19 2

2022-11-19 20:13:21,459 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_linearity_0025.fits'.
2022-11-19 20:13:21,460 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_mask_0038.fits'.
2022-11-19 20:13:21,460 - stpipe.Detector1Pipeline - INFO - Prefetch for PERSAT reference file is 'N/A'.
2022-11-19 20:13:21,461 - stpipe.Detector1Pipeline - INFO - Prefetch for READNOISE reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_readnoise_0017.fits'.
2022-11-19 20:13:21,462 - stpipe.Detector1Pipeline - INFO - Prefetch for REFPIX reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_refpix_0018.fits'.
2022-11-19 20:13:21,462 - stpipe.Detector1Pipeline - INFO - Prefetch for RESET reference file is 'N/A'.
2022-11-19 20:13:21,463 - stpipe.Detect

2022-11-19 20:14:02,855 - stpipe.Detector1Pipeline.linearity - WARNING - Keyword BAD_LIN_CORR does not correspond to an existing DQ mnemonic, so will be ignored
2022-11-19 20:14:03,503 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-11-19 20:14:03,619 - stpipe.Detector1Pipeline.dark_current - INFO - Step dark_current running with args (<RampModel(1, 15, 3200, 2048) from jw01128009001_03106_00001_nrs2_uncal.fits>,).
2022-11-19 20:14:03,621 - stpipe.Detector1Pipeline.dark_current - INFO - Step dark_current parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': 'stage1/', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/zen/projects/JWST_Observations_VHS1256b/reduction/nirspec_a_star/data', 'dark_output': None}
2022-11-19 20:14:03,809 - stpipe.Detector1Pipeline.dark_current - INFO - Using DARK reference fil

2022-11-19 20:16:30,222 - stpipe.Detector1Pipeline - INFO - ... ending calwebb_detector1
2022-11-19 20:16:30,223 - stpipe.Detector1Pipeline - INFO - Results used CRDS context: jwst_1015.pmap
2022-11-19 20:16:30,436 - stpipe.Detector1Pipeline - INFO - Saved model in stage1/jw01128009001_03106_00001_nrs2_rate.fits
2022-11-19 20:16:30,437 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline done
2022-11-19 20:16:30,449 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-11-19 20:16:30,450 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-11-19 20:16:30,452 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-11-19 20:16:30,453 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-11-19 20:16:30,454 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-11-19 20:16:30,456 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022

2022-11-19 20:16:30,873 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_gain_0020.fits'.
2022-11-19 20:16:30,873 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_ipc_0012.fits'.
2022-11-19 20:16:30,874 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_linearity_0025.fits'.
2022-11-19 20:16:30,874 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_mask_0038.fits'.
2022-11-19 20:16:30,875 - stpipe.Detector1Pipeline - INFO - Prefetch for PERSAT reference file is 'N/A'.
2022-11-19 20:16:30,875 - stpipe.Detector1Pipeline - INFO - Prefetch for READNOISE reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jw

...

2022-11-19 20:20:10,454 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/zen/projects/JWST_Observations_VHS1256b/reduction/nirspec_a_star/data'}
2022-11-19 20:20:10,475 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_linearity_0025.fits
2022-11-19 20:20:10,601 - stpipe.Detector1Pipeline.linearity - WARNING - Keyword BAD_LIN_CORR does not correspond to an existing DQ mnemonic, so will be ignored
2022-11-19 20:20:11,237 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-11-19 20:20:11,394 - stpipe.Detector1Pipeline.dark_current - INFO - Step dark_current running with args (<RampModel(1, 15, 320

2022-11-19 20:22:31,696 - stpipe.Detector1Pipeline.gain_scale - INFO - Rescaling by 1.0
2022-11-19 20:22:31,706 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale done
2022-11-19 20:22:31,941 - stpipe.Detector1Pipeline - INFO - Saved model in stage1/jw01128009001_03106_00003_nrs2_rateints.fits
2022-11-19 20:22:31,941 - stpipe.Detector1Pipeline - INFO - ... ending calwebb_detector1
2022-11-19 20:22:31,942 - stpipe.Detector1Pipeline - INFO - Results used CRDS context: jwst_1015.pmap
2022-11-19 20:22:32,154 - stpipe.Detector1Pipeline - INFO - Saved model in stage1/jw01128009001_03106_00003_nrs2_rate.fits
2022-11-19 20:22:32,155 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline done
2022-11-19 20:22:32,163 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-11-19 20:22:32,164 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-11-19 20:22:32,165 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance cr

2022-11-19 20:22:32,571 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_dark_0283.fits'.
2022-11-19 20:22:32,572 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_gain_0020.fits'.
2022-11-19 20:22:32,572 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_ipc_0012.fits'.
2022-11-19 20:22:32,573 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_linearity_0025.fits'.
2022-11-19 20:22:32,573 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_mask_0038.fits'.
2022-11-19 20:22:32,573 - stpipe.Detector1Pipeline - INFO - Prefetch for PERSAT re

2022-11-19 20:23:09,852 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/zen/projects/JWST_Observations_VHS1256b/reduction/nirspec_a_star/data'}
2022-11-19 20:23:09,873 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_linearity_0025.fits
2022-11-19 20:23:09,995 - stpipe.Detector1Pipeline.linearity - WARNING - Keyword BAD_LIN_CORR does not correspond to an existing DQ mnemonic, so will be ignored
2022-11-19 20:23:10,556 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-11-19 20:23:10,684 - stpipe.Detector1Pipeline.dark_current - INFO - Step dark_current running with args (<RampModel(1, 15, 320

2022-11-19 20:25:36,935 - stpipe.Detector1Pipeline.gain_scale - INFO - Rescaling by 1.0
2022-11-19 20:25:36,944 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale done
2022-11-19 20:25:37,150 - stpipe.Detector1Pipeline - INFO - Saved model in stage1/jw01128009001_03106_00004_nrs2_rateints.fits
2022-11-19 20:25:37,150 - stpipe.Detector1Pipeline - INFO - ... ending calwebb_detector1
2022-11-19 20:25:37,151 - stpipe.Detector1Pipeline - INFO - Results used CRDS context: jwst_1015.pmap
2022-11-19 20:25:37,348 - stpipe.Detector1Pipeline - INFO - Saved model in stage1/jw01128009001_03106_00004_nrs2_rate.fits
2022-11-19 20:25:37,348 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline done


In [9]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 4012.4003 seconds


In [10]:
# Look for input uncalibrated slope files from the Detector1 pipeline
sstring = 'stage1/'+ '*rate.fits'
rate_files = sorted(glob.glob(sstring))
print('Found ' + str(len(rate_files)) + ' input files to process')
print(rate_files)

Found 4 input files to process
['stage1/jw01128009001_03106_00001_nrs2_rate.fits', 'stage1/jw01128009001_03106_00002_nrs2_rate.fits', 'stage1/jw01128009001_03106_00003_nrs2_rate.fits', 'stage1/jw01128009001_03106_00004_nrs2_rate.fits']


In [11]:
## create association file for dithered data
call(["asn_from_list", "-o","l2_asn.json"] + rate_files + ["--product-name", "dither"])
dms = datamodels.open("l2_asn.json")

2022-11-19 21:20:11,904 - stpipe - WARNING - /Users/zen/opt/miniconda3/envs/jwst_env_1_8_2/lib/python3.10/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
  warnings.warn(err_str, UserWarning)

2022-11-19 21:20:12,006 - stpipe - WARNING - /Users/zen/opt/miniconda3/envs/jwst_env_1_8_2/lib/python3.10/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
  warnings.warn(err_str, UserWarning)



In [12]:
def runspec2(json_file):
    spec2 = Spec2Pipeline()
    ## Set some parameters that pertain to the entire pipeline
    spec2.save_results = True
    spec2.output_dir = spec2_dir
    spec2.extract_1d.skip = True
    spec2.run(json_file)

In [13]:
#runspec2('l2_asn.json')
runspec2(rate_files)

2022-11-19 21:20:12,278 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2022-11-19 21:20:12,280 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-11-19 21:20:12,281 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-11-19 21:20:12,282 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2022-11-19 21:20:12,283 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2022-11-19 21:20:12,284 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2022-11-19 21:20:12,286 - stpipe.Spec2Pipeline.master_background_mos - INFO - MasterBackgroundMosStep instance created.
2022-11-19 21:20:12,287 - stpipe.Spec2Pipeline.master_background_mos.flat_field - INFO - FlatFieldStep instance created.
2022-11-19 21:20:12,288 - stpipe.Spec2Pipeline.master_background_mos.pathloss - INFO - PathLossStep instance created.
2022-11-19 21:20:12,289 - stpipe.Spec2Pipeline.

2022-11-19 21:20:12,651 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'jw01128009001_03106_00001_nrs2_rate.fits' reftypes = ['area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'mrsxartcorr', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'sflat', 'specwcs', 'wavecorr', 'wavelengthrange', 'wfssbkg']
2022-11-19 21:20:12,657 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_area_0047.fits'.
2022-11-19 21:20:12,657 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2022-11-19 21:20:12,657 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_camera_0007.asdf'.
2022-11-19 21:20:12,658 - stpipe.Spec2Pipeline - INF

2022-11-19 21:20:12,687 - stpipe.Spec2Pipeline - INFO - Prefetch for IFUSLICER reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_ifuslicer_0010.asdf'.
2022-11-19 21:20:12,687 - stpipe.Spec2Pipeline - INFO - Prefetch for MRSXARTCORR reference file is 'N/A'.
2022-11-19 21:20:12,688 - stpipe.Spec2Pipeline - INFO - Prefetch for MSA reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_msa_0008.asdf'.
2022-11-19 21:20:12,688 - stpipe.Spec2Pipeline - INFO - Prefetch for MSAOPER reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_msaoper_0003.json'.
2022-11-19 21:20:12,689 - stpipe.Spec2Pipeline - INFO - Prefetch for OTE reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_ote_0008.asdf'.
2022-11-19 21:20:12,689 - stpipe.Spec2Pipeline - INFO - Prefetch for PATHLOSS reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_pathl

2022-11-19 21:20:12,720 - stpipe.Spec2Pipeline - INFO - Prefetch for CUBEPAR reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_cubepar_0003.fits'.
2022-11-19 21:20:12,720 - stpipe.Spec2Pipeline - INFO - Prefetch for DFLAT reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_dflat_0002.fits'.
2022-11-19 21:20:12,721 - stpipe.Spec2Pipeline - INFO - Prefetch for DISPERSER reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_disperser_0057.asdf'.
2022-11-19 21:20:12,721 - stpipe.Spec2Pipeline - INFO - Prefetch for DISTORTION reference file is 'N/A'.
2022-11-19 21:20:12,722 - stpipe.Spec2Pipeline - INFO - Prefetch for DRIZPARS reference file is 'N/A'.
2022-11-19 21:20:12,722 - stpipe.Spec2Pipeline - INFO - Prefetch for FFLAT reference file is '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_fflat_0036.fits'.
2022-11-19 21:20:12,722 - stpipe.Spec2Pipeline - INFO - Pref

2022-11-19 21:20:21,780 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': ''}
2022-11-19 21:20:21,781 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step skipped.
2022-11-19 21:20:21,783 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract done
2022-11-19 21:20:21,890 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging running with args (<IFUImageModel(2048, 2048) from jw01128009001_03106_00001_nrs2_rate.fits>,).
2022-11-19 21:20:21,891 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_result

...

2022-11-19 21:40:07,689 - stpipe.Spec2Pipeline.assign_wcs - INFO - gwa_xtilt is 0.3532653150000012 deg
2022-11-19 21:40:07,690 - stpipe.Spec2Pipeline.assign_wcs - INFO - theta_y correction: 0.0018865959647736948 deg
2022-11-19 21:40:07,691 - stpipe.Spec2Pipeline.assign_wcs - INFO - theta_x correction: 0.00011620025040863247 deg
2022-11-19 21:40:07,769 - stpipe.Spec2Pipeline.assign_wcs - INFO - Applied Barycentric velocity correction : 1.000000367120778
2022-11-19 21:40:10,978 - stpipe.Spec2Pipeline.assign_wcs - INFO - Created a NIRSPEC nrs_ifu pipeline with references {'distortion': None, 'filteroffset': None, 'specwcs': None, 'regions': None, 'wavelengthrange': '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_wavelengthrange_0006.asdf', 'camera': '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_camera_0007.asdf', 'collimator': '/Users/zen/crds_cache/jwst_ops/references/jwst/nirspec/jwst_nirspec_collimator_0007.asdf', 'disperser': '/Users/zen/crd

2022-11-19 21:46:20,428 - stpipe.Spec2Pipeline.fringe - INFO - Step fringe parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': 'fringe', 'search_output_file': True, 'input_dir': ''}
2022-11-19 21:46:20,429 - stpipe.Spec2Pipeline.fringe - INFO - Step skipped.
2022-11-19 21:46:20,431 - stpipe.Spec2Pipeline.fringe - INFO - Step fringe done
2022-11-19 21:46:20,796 - stpipe.Spec2Pipeline.pathloss - INFO - Step pathloss running with args (<IFUImageModel(2048, 2048) from jw01128009001_03106_00003_nrs2_rate.fits>,).
2022-11-19 21:46:20,797 - stpipe.Spec2Pipeline.pathloss - INFO - Step pathloss parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'pathloss', 'search_output

2022-11-19 21:50:00,498 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': True, 'suffix': 'x1d', 'search_output_file': True, 'input_dir': '', 'smoothing_length': None, 'bkg_fit': None, 'bkg_order': None, 'bkg_sigma_clip': 3.0, 'log_increment': 50, 'subtract_background': None, 'use_source_posn': None, 'center_xy': None, 'apply_apcorr': True, 'soss_threshold': 0.01, 'soss_n_os': 2, 'soss_transform': None, 'soss_tikfac': None, 'soss_width': 40.0, 'soss_bad_pix': 'model', 'soss_modelname': None}
2022-11-19 21:50:00,498 - stpipe.Spec2Pipeline.extract_1d - INFO - Step skipped.
2022-11-19 21:50:00,501 - stpipe.Spec2Pipeline.extract_1d - INFO - Step extract_1d done
2022-11-19 21:50:00,501 - stpipe.Spec2Pipeline - INFO - Finished processing product stage1/jw01128009001_03106_00003_nrs2
2022

2022-11-19 21:50:26,425 - stpipe.Spec2Pipeline.srctype - INFO - Input EXP_TYPE is NRS_IFU
2022-11-19 21:50:26,425 - stpipe.Spec2Pipeline.srctype - INFO - Input SRCTYAPT = UNKNOWN
2022-11-19 21:50:26,426 - stpipe.Spec2Pipeline.srctype - INFO - Exposure is nodded; setting SRCTYPE = POINT
2022-11-19 21:50:26,429 - stpipe.Spec2Pipeline.srctype - INFO - Step srctype done
2022-11-19 21:50:26,795 - stpipe.Spec2Pipeline.flat_field - INFO - Step flat_field running with args (<IFUImageModel(2048, 2048) from jw01128009001_03106_00004_nrs2_rate.fits>,).
2022-11-19 21:50:26,797 - stpipe.Spec2Pipeline.flat_field - INFO - Step flat_field parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'flat_field', 'search_output_file': True, 'input_dir': '', 'save_interpolated_flat': False, 'user_supplied_flat': None, 'inverse': False}
2022-11-19 21:

2022-11-19 21:59:10,260 - stpipe.Spec2Pipeline.cube_build - INFO - axis#  Naxis  CRPIX    CRVAL      CDELT(arcsec)  Min & Max (xi, eta arcsec)
2022-11-19 21:59:10,261 - stpipe.Spec2Pipeline.cube_build - INFO - Axis 1    49  25.00 -87.85543541   0.10000000  -2.45000004   2.45000004
2022-11-19 21:59:10,261 - stpipe.Spec2Pipeline.cube_build - INFO - Axis 2    51  26.00  69.45794808   0.10000000  -2.55000004   2.55000004
2022-11-19 21:59:10,261 - stpipe.Spec2Pipeline.cube_build - INFO - axis#  Naxis  CRPIX    CRVAL      CDELT(microns)  Min & Max (microns)
2022-11-19 21:59:10,262 - stpipe.Spec2Pipeline.cube_build - INFO - Axis 3  1910   1.00   2.41972531   0.00039600   2.41952731   3.17588729
2022-11-19 21:59:10,262 - stpipe.Spec2Pipeline.cube_build - INFO - Cube covers grating, filter: g235h, f170lp
2022-11-19 21:59:10,262 - stpipe.Spec2Pipeline.cube_build - INFO - Output Name: stage2/jw01128009001_03106_00004_nrs2_s3d.fits
2022-11-19 21:59:14,410 - stpipe.Spec2Pipeline.cube_build - INFO -

In [14]:
### Calling in the calibrated detector images produced by running the stage 2 pipeline on the modified NRS fits files
cal_files = glob.glob('stage2/*cal.fits')
print(cal_files)

['stage2/jw01128009001_03106_00004_nrs2_cal.fits', 'stage2/jw01128009001_03106_00003_nrs2_cal.fits', 'stage2/jw01128009001_03106_00002_nrs2_cal.fits', 'stage2/jw01128009001_03106_00001_nrs2_cal.fits']


In [15]:
### Calling in the calibrated detector images produced by running the stage 2 pipeline on the modified NRS fits files
cal_files = glob.glob('stage2/*cal.fits')
print(cal_files)

['stage2/jw01128009001_03106_00004_nrs2_cal.fits', 'stage2/jw01128009001_03106_00003_nrs2_cal.fits', 'stage2/jw01128009001_03106_00002_nrs2_cal.fits', 'stage2/jw01128009001_03106_00001_nrs2_cal.fits']


In [16]:
## create association file for dithered data. Make a json file for each dither set.
call(["asn_from_list", "-o","l3_asn.json"] + cal_files + ["--product-name", "dither"])

2022-11-19 22:00:58,503 - stpipe - WARNING - /Users/zen/opt/miniconda3/envs/jwst_env_1_8_2/lib/python3.10/site-packages/jwst/associations/association.py:215: UserWarning: 'expname' contains path, but should only be a filename.  All input files should be in a single directory, so no path is needed.
  warnings.warn(err_str, UserWarning)



0

In [17]:
#cube_data = fits.getdata('stage2/jw01128009001_03106_00001_nrs2_s3d.fits')
#cube=SpectralCube.read('stage2/jw01128009001_03106_00001_nrs2_s3d.fits',hdu=1)

cube_data = fits.getdata('stage3/dither_g235h-f170lp_s3d.fits')
cube=SpectralCube.read('stage3/dither_g235h-f170lp_s3d.fits',hdu=1)


wavelength=cube.spectral_axis
wavelength=n.array(wavelength)

cube_collapse = n.nanmedian(cube_data, axis = 0)

FileNotFoundError: [Errno 2] No such file or directory: 'stage3/dither_g235h-f170lp_s3d.fits'

## Collapsed Image of Cube

In [ ]:
%matplotlib notebook
plt.figure('collapsed cube')
plt.imshow((cube_collapse))#, vmin = 0, vmax = 3.8047e-11)
plt.show()

In [ ]:
## fitting a gaussian to the collapsed cube which is a median image to remove high pixels
image = cube_collapse
y_len, x_len = image.shape
bounds = ((0, 0, n.nanmin(image), 1, 1, n.nanmedian(image)) , (x_len, y_len, n.nanmax(image), x_len, y_len, 2 * n.nanmax(image)) )
guess = n.array([34, 38, n.nanmin(image), 1.2, 1.2, n.nanmax(image)])
fit_arr = optimize.least_squares(image_solve, guess, args = (image,), bounds = bounds )#, bounds=bounds)
x0_fit, y0_fit, offset_fit, sigma_x_fit, sigma_y_fit, A_fit = fit_arr.x


x0_fit = n.round(x0_fit, decimals = 0) #rounding because pipeline does not take float values
y0_fit = n.round(y0_fit, decimals = 0) #rounding because pipeline does not take float values


print('centroids (x, y)')
print(x0_fit, y0_fit)
print('sigmax, sigma y')
print(sigma_x_fit, sigma_y_fit)

In [ ]:
def runspec3(json_file):
# This initial setup is just to make sure that we get the latest parameter reference files
# pulled in for our files.  This is a temporary workaround to get around an issue with
# how this pipeline calling method works.
    crds_config = Spec3Pipeline.get_config_from_reference(json_file)# The exact asn file used doesn't matter
    spec3 = Spec3Pipeline.from_config_section(crds_config)
    
    spec3.output_dir = spec3_dir
    spec3.output_type = 'band'
    spec3.outlier_detection.skip = True
    #spec3.cube_build.skip = True YOU CANT EXTRACT WITHOUT BUILDING THE CUBE 
    #spec3.extract_1d.center_xy = (int(x0_fit), int(y0_fit))
    spec3.extract_1d.center_xy = (int(x0_fit), int(y0_fit))
    spec3.save_results = True
    
    spec3(json_file)

In [ ]:
runspec3('l3_asn.json')

In [ ]:
data = fits.getdata('stage3/dither_g235h-f170lp_x1d.fits')
wavelength = data['wavelength']
flux  = data['flux']

%matplotlib notebook
plt.ylim([0,.1])
plt.plot(wavelength, flux, label = 'stage 3')
plt.legend()